In [1]:
import numpy as np
from math import factorial as fact
from functools import lru_cache
from itertools import product

from utils import * #Contains relevant functions for A(d) such as norm and cardinality
from tqdm.notebook import tqdm

#### First define a function which can compute the coefficient of a cycle $[\gamma_\ell]$ in any product $a\cdot b$ in $A(d)$

In [2]:
@lru_cache(maxsize=1000000) # use memoization since small values will be called repeatedly.
def Lambda(l: int,a: tuple,b: tuple):
    '''
    Computes normalized coefficient of l-cycle in the product a*b.
    '''
    a_arr, b_arr = np.array(a),np.array(b) # convert to arrays in order to apply standard functions
    
    # base case
    if card(a_arr)==0 or card(b_arr)==0 or l<2:
        return fact(l-1)
    
    # recursion formula
    if norm(a_arr)+norm(b_arr)==l-1:
        if magn(a_arr)<l and magn(b_arr)<=l:
            answer = 0
            for i in range(2,min(l+1,len(b)+2)):
                if b[i-2] > 0:
                    answer += (i-1)*partialterm(l,i,b_arr)*Lambda(l-1,a,tuple(partial(i,b)))
            return l*answer // (l-magn(a_arr))

        elif magn(a_arr)<=l and magn(b_arr)<l:
            answer = 0
            for i in range(2,min(l+1,len(a)+2)):
                if a[i-2] > 0:
                    answer += (i-1)*partialterm(l,i,a_arr)*Lambda(l-1,b,tuple(partial(i,a)))
            return l*answer//(l-magn(b_arr))
    return 0

# non-normalized coefficient of l-cycle in the product a*b
cycle_coeff = lambda l,a,b: Lambda(l,a,b)//fact(l-1)

In [3]:
## lambda(9,[2,..],[1,0,0,0,1,...]) = 54
cycle_coeff(9,
            (2,0,0,0,0,0),
            (1,0,0,0,1,0))

54

#### Now extend coefficients from $\ell$-cycles to general cycle types. 

In [4]:
def gen_cdecomp(a: tuple,c: tuple):
    '''
    Generates all c-decompositions of a as a list of pairs (A,A').
    '''
    answer = []
    a_arr = np.array(a)
    nu = np.nonzero(c)[0][0] +2 #smallest non-zero index
    
    # Create a generator for possible values of A:
    intervals = [range(0,min(a[i],nu)*(i<nu)+1) for i in range(0,len(a))]
    cart_prod = product(*intervals)
    
    
    for A in cart_prod:
        A_arr = np.array(A)
        if magn(A_arr)<=nu:
            answer.append((A,tuple(a_arr-A_arr)))
        
    return answer

In [5]:
gen_cdecomp((0,1,2,3),(0,0,2,1))

[((0, 0, 0, 0), (0, 1, 2, 3)),
 ((0, 0, 1, 0), (0, 1, 1, 3)),
 ((0, 1, 0, 0), (0, 0, 2, 3))]

Now define a general procedure for computing coefficients

In [6]:
@lru_cache(maxsize=1000000)
def Lambda2(c,a,b):
    '''
    Computes coefficient of $[c]$ in product $[a]\cdot [b]$
    '''
    if norm(a) + norm(b) != norm(c):
        return 0
    
    if sum(a)==0 and sum(b)==0 and sum(c) == 0:
        return 1

    nu = np.nonzero(c)[0][0] +2
    a_decomps = gen_cdecomp(a,c)
    b_decomps = gen_cdecomp(b,c)
    
    answer = 0
    for A1,A2 in a_decomps:
        for B1,B2 in b_decomps:
            if norm(np.array(A1))+norm(np.array(B1)) == nu-1:
                answer += cycle_coeff(nu,A1,B1) * Lambda2(tuple(c - np.eye(1,len(c),nu-2,dtype=int)[0]), A2,B2)
    return answer

In [7]:
### lambda([1,1,1], [3,0,...], [3,0,...]) = 12
Lambda2((1,1,1),(3,),(3,))

12

## Generate tables of monomials

To compute the product $a\cdot b$ of basis elements we cycle through all cycletypes of relevant norm and compute the coefficient.

In [8]:
def multiply_basis(a: tuple,b: tuple):
    '''
    Given two basis elements a and b, compute the product a*b as a dictionary.
    '''
    product = {}
    
    for cycle_type in generate_cycletypes(norm(a)+norm(b)):
        if coeff := Lambda2(cycle_type,a,b):
            product[cycle_type] = coeff
    
    return product

In [9]:
print("Multiply x*y:", "\n",
      multiply_basis((1,),(2,)))

print("Multiply y*z:", "\n",
      multiply_basis((2,),(3,)))

print("multiply gamma_3 and gamma_4:", "\n",
     multiply_basis((0,1),(0,0,1)))

Multiply x*y: 
 {(3,): 3, (1, 1): 3, (0, 0, 1): 2}
Multiply y*z: 
 {(5,): 10, (3, 1): 9, (2, 0, 1): 6, (1, 2): 9, (1, 0, 0, 1): 5, (0, 1, 1): 6, (0, 0, 0, 0, 1): 3}
multiply gamma_3 and gamma_4: 
 {(0, 1, 1): 1, (0, 0, 0, 0, 1): 6}


#### Once we can multiply basis elements we can extend linearly to general elements

In [10]:
def multiply_generator(generator: tuple, g: dict, d: int=100):
    product = {}
    
    for basis1, coeff1 in g.items():
        for basis2, coeff2 in multiply_basis(generator,basis1).items():
            if magn(basis2) > d:
                pass
            elif basis2 in product.keys():
                product[basis2] += coeff1*coeff2
            else:
                product[basis2] = coeff1*coeff2
    return product

In [11]:
print("Compute xy^2 as y * xy:")
print(multiply_generator((2,), {(3,): 3, (1, 1): 3, (0, 0, 1): 2}))

print("\n")
print("Multiply gamma_3 with gamma_3 and gamma_4:")
print(multiply_generator((0,1), {(0, 1, 1): 1, (0, 0, 0, 0, 1): 6}))

Compute xy^2 as y * xy:
{(5,): 30, (3, 1): 36, (2, 0, 1): 44, (1, 2): 45, (1, 0, 0, 1): 55, (0, 1, 1): 60, (0, 0, 0, 0, 1): 81}


Multiply gamma_3 with gamma_3 and gamma_4:
{(0, 2, 1): 2, (0, 1, 0, 0, 1): 12, (0, 0, 1, 1): 5, (0, 0, 0, 0, 0, 0, 1): 64}


#### Define a procedure for generating all monomials

In [12]:
def generate_monomials(d=12,use_cycles=False):
    # Whether to use cycle generators or transpositions as generators.
    if use_cycles:
        generators = [(c,(0,)*i + (1,)) for i,c in enumerate(alphabet[0:d//2])] #cycle generators
    else:
        generators = [(c,(i+1,)) for i,c in enumerate(alphabet[0:d//2])] #(a_2,0,0,...)
    
    # The list of monomials is initialized as a list of the generators
    monomials = [(i,{j:1}) for i,j in generators]
    uniquenames = set([c for c in alphabet[0:d//2]])
    
    # Repeatedly multiply each generator on list of monomials and add to list of monomials
    for name1, generator in tqdm(generators):
        for name2,g in monomials:
            if norm(list(g.keys())[0]) + norm(generator) <= d:
                newname = "".join(sorted(name1+name2))
                if newname not in uniquenames:
                    uniquenames.add(newname)
                    prod = multiply_generator(generator, g,d)
                    if prod != {}:
                        monomials.append((newname, prod))

    return monomials

In [13]:
use_cycles = True
monomials = generate_monomials(d = 15, use_cycles=use_cycles)
print(len(monomials))
monomials[0:24]


432


[('a', {(1,): 1}),
 ('b', {(0, 1): 1}),
 ('c', {(0, 0, 1): 1}),
 ('d', {(0, 0, 0, 1): 1}),
 ('e', {(0, 0, 0, 0, 1): 1}),
 ('f', {(0, 0, 0, 0, 0, 1): 1}),
 ('g', {(0, 0, 0, 0, 0, 0, 1): 1}),
 ('aa', {(2,): 2, (0, 1): 3}),
 ('ab', {(1, 1): 1, (0, 0, 1): 4}),
 ('ac', {(1, 0, 1): 1, (0, 0, 0, 1): 5}),
 ('ad', {(1, 0, 0, 1): 1, (0, 0, 0, 0, 1): 6}),
 ('ae', {(1, 0, 0, 0, 1): 1, (0, 0, 0, 0, 0, 1): 7}),
 ('af', {(1, 0, 0, 0, 0, 1): 1, (0, 0, 0, 0, 0, 0, 1): 8}),
 ('ag', {(1, 0, 0, 0, 0, 0, 1): 1, (0, 0, 0, 0, 0, 0, 0, 1): 9}),
 ('aaa', {(3,): 6, (1, 1): 9, (0, 0, 1): 16}),
 ('aab', {(2, 1): 2, (1, 0, 1): 8, (0, 2): 6, (0, 0, 0, 1): 25}),
 ('aac', {(2, 0, 1): 2, (1, 0, 0, 1): 10, (0, 1, 1): 3, (0, 0, 0, 0, 1): 36}),
 ('aad',
  {(2, 0, 0, 1): 2,
   (1, 0, 0, 0, 1): 12,
   (0, 1, 0, 1): 3,
   (0, 0, 0, 0, 0, 1): 49}),
 ('aae',
  {(2, 0, 0, 0, 1): 2,
   (1, 0, 0, 0, 0, 1): 14,
   (0, 1, 0, 0, 1): 3,
   (0, 0, 0, 0, 0, 0, 1): 64}),
 ('aaf',
  {(2, 0, 0, 0, 0, 1): 2,
   (1, 0, 0, 0, 0, 0, 1): 16,


#### Format the monomials and sort them by norm

In [14]:
## Create sorted dictionary of monomials
sorted_monoms = {i:[] for i in range(1,16)}
for name, dic in monomials:
    n = get_norm(name)
    sorted_monoms[n] += [(name, dic)]

In [15]:
## Get all monomials of norm 5
sorted_monoms[5]

[('e', {(0, 0, 0, 0, 1): 1}),
 ('ad', {(1, 0, 0, 1): 1, (0, 0, 0, 0, 1): 6}),
 ('aac', {(2, 0, 1): 2, (1, 0, 0, 1): 10, (0, 1, 1): 3, (0, 0, 0, 0, 1): 36}),
 ('aaab',
  {(3, 1): 6,
   (2, 0, 1): 24,
   (1, 2): 18,
   (1, 0, 0, 1): 75,
   (0, 1, 1): 52,
   (0, 0, 0, 0, 1): 216}),
 ('aaaaa',
  {(5,): 120,
   (3, 1): 180,
   (2, 0, 1): 320,
   (1, 2): 270,
   (1, 0, 0, 1): 625,
   (0, 1, 1): 480,
   (0, 0, 0, 0, 1): 1296}),
 ('bc', {(0, 1, 1): 1, (0, 0, 0, 0, 1): 6}),
 ('abb', {(1, 2): 2, (1, 0, 0, 1): 5, (0, 1, 1): 8, (0, 0, 0, 0, 1): 36})]

## Compute relations

In [16]:
from sympy import *

In [17]:
# Define a function which eliminates elements with too large support and finds relations
def relations(d=9, use_cycles = False):
    '''
    Returns the relations between monomials of generators for each norm
    '''
    
    # Create a string of allowed symbols in monomial names
    if use_cycles:
        allowed_generators = alphabet[0:d//2] + "^1234567890" #Generators that vanish in lower dimension d.
    else:
        allowed_generators = alphabet[0:d//2] + "^1234567890" #Generators that vanish in lower dimension d.
    
    output = []
    
    for n in range(2,d+1):
        key_names = set([])
        monoms = []

        # Get the relevant monomials and basis elements in A(d) of norm n.
        for name, dic in sorted_monoms[n]:
            ## Consider only monomials of _non-zero_ generators:
            monom_non_triv = True
            for char in name:
                if char not in allowed_generators:
                    monom_non_triv = False
            if monom_non_triv:
                monoms.append((name,dic))
                # Take all summands with valid support
                for element in dic.keys():
                    if magn(element) <= d: #Sort out elements with support equal to d.
                        key_names.add(element)
        key_names = sorted(list(key_names))
        monoms = sorted(monoms)
        
        # Create a matrix with coefficients from the monomial equations
        A = np.zeros((len(key_names),len(monoms)), dtype = int)

        for i, key in enumerate(key_names):
            for j, equation in enumerate(monoms):
                if key in (equation[1]).keys():
                    A[i,j] = (equation[1])[key]
        
        output.append(([m[0] for m in monoms], Matrix(A).nullspace()))
    return output

In [18]:
def relations_latex_output(d=8, use_cycles = False):
    L = relations(d, use_cycles)
    for n, (names, null_space) in enumerate(L):
        if len(null_space) and n+2!=d:
            print("Norm", n+2)
            print(r"\[ \begin{bmatrix}")
            s = ""
            for x in names:
                s += format_monomname(x) + " & "
            print(s[:-2] + r"\\")
            for i,row in enumerate(null_space):
                rel = ""
                common_denom = ilcm(*tuple(x.q for x in row))
                for x in common_denom * row:
                    rel += str(x) + " & "
                print(rel[:-2] + r"\\")
            print(r"\end{bmatrix} \]")
        elif n+2==d:
            print("monomials in norm", n+2)
            print(r"\[", "\n",
                  "".join(format_monomname(x)+", " for x in names)[:-2], "\n",
                  r"\]")

In [19]:
for i in range(4,11):
    print(r"\section*{relations in A("+str(i)+")}")
    relations_latex_output(i, use_cycles)
    print("")

\section*{relations in A(4)}
Norm 3
\[ \begin{bmatrix}
a^{3} & ab \\
-1 & 4 \\
\end{bmatrix} \]
monomials in norm 4
\[ 
 a^{4}, a^{2}b, b^{2} 
 \]

\section*{relations in A(5)}
Norm 4
\[ \begin{bmatrix}
a^{4} & a^{2}b & b^{2} \\
-1 & 5 & 0 \\
-1 & 0 & 25 \\
\end{bmatrix} \]
monomials in norm 5
\[ 
 a^{5}, a^{3}b, ab^{2} 
 \]

\section*{relations in A(6)}
Norm 4
\[ \begin{bmatrix}
a^{4} & a^{2}b & ac & b^{2} \\
1 & -11 & 24 & 6 \\
\end{bmatrix} \]
Norm 5
\[ \begin{bmatrix}
a^{5} & a^{3}b & a^{2}c & ab^{2} & bc \\
-1 & 6 & 0 & 0 & 0 \\
-1 & 0 & 36 & 0 & 0 \\
-1 & 0 & 0 & 36 & 0 \\
-1 & 0 & 0 & 0 & 216 \\
\end{bmatrix} \]
monomials in norm 6
\[ 
 a^{6}, a^{4}b, a^{3}c, a^{2}b^{2}, abc, b^{3}, c^{2} 
 \]

\section*{relations in A(7)}
Norm 5
\[ \begin{bmatrix}
a^{5} & a^{3}b & a^{2}c & ab^{2} & bc \\
1 & -13 & 28 & 14 & 0 \\
11 & -129 & 280 & 0 & 588 \\
\end{bmatrix} \]
Norm 6
\[ \begin{bmatrix}
a^{6} & a^{4}b & a^{3}c & a^{2}b^{2} & abc & b^{3} & c^{2} \\
-1 & 7 & 0 & 0 & 0 & 0 & 0 \\
-1 &

In [20]:
def compute_minimal_relations(d):
    sufficient_relations = relations(d)[(d-1)//2:]
    
    name_tables = []
    lifted_relations = []
    for names,A in sufficient_relations:
        name_tables.append({name:i for i,name in enumerate(names)})
        lifted_relations.append(Matrix(len(names),0,[]))
    
    for i,char in enumerate(alphabet):
        for j, (names_lower, rels) in enumerate(sufficient_relations):
            if i+j+1 < len(sufficient_relations):
                names_higher = name_tables[i+j+1]
                conversion_matrix = zeros(len(names_higher),len(names_lower))

                for x,name in enumerate(names_lower):
                    new_name = "".join(sorted(name+char))
                    if new_name in names_higher:
                        conversion_matrix[names_higher[new_name],x] = 1
                
                rels_lifted = []
                for rel in rels:
                    rels_lifted.append(conversion_matrix*rel)
                lifted_relations[i+j+1] = lifted_relations[i+j+1].row_join(Matrix([rels_lifted]))
    
    L = []
    for (m,A),B in zip(sufficient_relations,lifted_relations):
        L.append(len(A)-B.rank())
    print("relations from norm", (d+3)//2, "\n", L, ", sum:", sum(L))

In [21]:
for d in range(3,11):
    print("A({}):".format(d))
    compute_minimal_relations(d)

A(3):
relations from norm 3 
 [1] , sum: 1
A(4):
relations from norm 3 
 [1, 2] , sum: 3
A(5):
relations from norm 4 
 [2, 1] , sum: 3
A(6):
relations from norm 4 
 [1, 3, 2] , sum: 6
A(7):
relations from norm 5 
 [2, 4, 1] , sum: 7
A(8):
relations from norm 5 
 [1, 4, 4, 2] , sum: 11
A(9):
relations from norm 6 
 [2, 5, 5, 1] , sum: 13
A(10):
relations from norm 6 
 [1, 4, 7, 4, 2] , sum: 18
